In [1]:
import pathlib
from math import exp, pow
from PIL import Image
import numpy as np
import matplotlib.pyplot
import matplotlib
from ctypes import *

class IdealNotchFilter:
    def __init__(self):
        pass
    
    def apply_filter(self, fshift, points, d0, path):
        m = fshift.shape[0]
        n = fshift.shape[1]
        for u in range(m):
            for v in range(n):
                for d in range(len(points)):
                    u0 = points[d][0]
                    v0 = points[d][1]
                    u0, v0 = v0, u0
                    d1 = pow(pow(u - u0, 2) + pow(v - v0, 2), 1)
                    d2 = pow(pow(u + u0, 2) + pow(v + v0, 2), 1)
                    if d1 <= d0 or d2 <= d0:
                        fshift[u][v] *= 0.0
        f_ishift = np.fft.ifftshift(fshift)
        img_back = np.fft.ifft2(f_ishift)
        img_back = np.abs(img_back)
        matplotlib.image.imsave(path, img_back, cmap = "gray")
        return

class ButterworthNotchFilter:
    def __init__(self):
        pass
    
    def apply_filter(self, fshift, points, d0, path, order = 1):
        m = fshift.shape[0]
        n = fshift.shape[1]
        for u in range(m):
            for v in range(n):
                for d in range(len(points)):
                    u0 = points[d][0]
                    v0 = points[d][1]
                    u0, v0 = v0, u0
                    d1 = pow(pow(u - u0, 2) + pow(v - v0, 2), 0.5)
                    d2 = pow(pow(u + u0, 2) + pow(v + v0, 2), 0.5)
                    fshift[u][v] *= (1.0 / (1 + pow((d0 * d0) / (d1 * d2), order))) 
                    
        f_ishift = np.fft.ifftshift(fshift)
        img_back = np.fft.ifft2(f_ishift)
        img_back = np.abs(img_back)
        matplotlib.image.imsave(path, img_back, cmap = "gray")
        return
    
class GaussianNotchFilter:
    def __init__(self):
        pass
    
    def apply_filter(self, fshift, points, d0, path):
        m = fshift.shape[0]
        n = fshift.shape[1]
        for u in range(m):
            for v in range(n):
                for d in range(len(points)):
                    u0 = points[d][0]
                    v0 = points[d][1]
                    u0, v0 = v0, u0
                    d1 = pow(pow(u - u0, 2) + pow(v - v0, 2), 0.5)
                    d2 = pow(pow(u + u0, 2) + pow(v + v0, 2), 0.5)
                    fshift[u][v] *= (1 - exp(-0.5 * (d1 * d2 / pow(d0, 2))))

        f_ishift = np.fft.ifftshift(fshift)
        img_back = np.fft.ifft2(f_ishift)
        img_back = np.abs(img_back)
        matplotlib.image.imsave(path, img_back, cmap = "gray")
        return


In [2]:
def get_fshift_and_save_dft():
    img = Image.open(pathlib.Path("images\\duck.jpg"))
    img = np.asarray(img)
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    dft = 20 * np.log(np.abs(fshift))
    matplotlib.image.imsave(pathlib.Path("images\\dft_duck.png"), dft, cmap = "gray")
    return fshift, dft

In [3]:
fshift, dft = get_fshift_and_save_dft()
points = [  [239.96445273,255.77032735],
            [250.77754671,250.36378036],
            [259.96867659,260.63621964],
            [271.32242527,255.22967265]  ] # Duck photo
IdealNotchFilter().apply_filter(fshift, points, float(3), pathlib.Path("images\\ideal_notch_filtered_duck_img.png"))
        
ButterworthNotchFilter().apply_filter(fshift, points, float(3), pathlib.Path("images\\butterworth_notch_filtered_duck_img.png"), order = 1)
         
GaussianNotchFilter().apply_filter(fshift, points, float(3), pathlib.Path("images\\gaussian_notch_filtered_duck_img.png.png"))
            